In [ ]:
pip install "openai>=1.76.0" "pydantic>=2.10,<3" "griffe>=1.5.6,<2" "typing-extensions>=4.12.2,<5" "requests>=2.0,<3" "types-requests>=2.0,<3" "beautifulsoup4" "reportlab" "mcp>=1.6.0,<2; python_version >= '3.10'"

In [ ]:
pip install openai-agents

In [1]:
from __future__ import annotations

import asyncio
from dataclasses import dataclass
from typing import Literal

from agents import Agent, ItemHelpers, Runner, TResponseInputItem, trace

In [2]:
story_outline_generator = Agent(
    name="story_outline_generator",
    instructions=(
        "You generate a very short story outline based on the user's input."
        "If there is any feedback provided, use it to improve the outline."
    ),
)

In [3]:
@dataclass
class EvaluationFeedback:
    feedback: str
    score: Literal["pass", "needs_improvement", "fail"]

In [5]:
evaluator = Agent[None](
    name="evaluator",
    instructions=(
        "You evaluate a story outline and decide if it's good enough."
        "If it's not good enough, you provide feedback on what needs to be improved."
        "Never give it a pass on the first try."
    ),
    output_type=EvaluationFeedback,
)

In [6]:
msg = input("What kind of story would you like to hear? ")
input_items: list[TResponseInputItem] = [{"content": msg, "role": "user"}]

latest_outline: str | None = None
max_attempts = 4
attempt = 0

with trace("LLMJudge"):
    while attempt < max_attempts:
        story_outline_result = await Runner.run(
            story_outline_generator,
            input_items,
        )

        input_items = story_outline_result.to_input_list()
        latest_outline = ItemHelpers.text_message_outputs(story_outline_result.new_items)
        print("Story outline generated")

        evaluator_result = await Runner.run(evaluator, input_items)
        result: EvaluationFeedback = evaluator_result.final_output

        print(f"Evaluator score: {result.score}")

        if result.score == "pass":
            print("Story outline is good enough, exiting.")
            break

        print("Re-running with feedback")

        input_items.append({"content": f"Feedback: {result.feedback}", "role": "user"})

        attempt += 1

print(f"Final story outline: {latest_outline}")

What kind of story would you like to hear?  Wygeneruj historię o Stefanie Batorym, który spotkał smoka wawelskiego


Story outline generated
Evaluator score: needs_improvement
Re-running with feedback
Story outline generated
Evaluator score: needs_improvement
Re-running with feedback
Story outline generated
Evaluator score: needs_improvement
Re-running with feedback
Story outline generated
Evaluator score: needs_improvement
Re-running with feedback
Final story outline: **Tytuł: Stefan Batory i Sekrety Smoka Wawelskiego**

**Zarys fabuły:**

1. **Wprowadzenie:**
   - **Charakterystyka Stefana Batory:** Syn węgierskiego szlachcica, Stefan jest erudycyjny i ambitny, z młodzieńczą fascynacją legendami, które opowiadały o Smoku Wawelskim. Postrzega je jako klucz do historycznej mądrości i potęgi, co jest niezbędne do utrzymania silnego panowania w Polsce.

2. **Motywacja:**
   - **Chęć mądrości:** Stefan pragnie zrozumieć tajemnice Smoka, wierząc, że odkrycie jego sekretów wzmocni jego władzę i pozycję króla.
   - **Konflikty wewnętrzne:** Narastające napięcia z polską szlachtą, niechętni reformom, prowad